In [1]:
import os
import numpy as np
import pickle
from tensorrtserver.api import *
import cv2
import time
from glob import glob
from tqdm import tqdm

In [2]:
def get_img(infile):
    img_raw = cv2.imread(infile, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img_raw, (1024,1024))
    img = np.array(img, dtype=np.float32)
    img = img/255
    img = np.expand_dims(img, 0)
    return img

In [3]:
def infer_with_trt(test_file, service_ip, service_port, service_name, out_dir):
    protocol = ProtocolType.from_str('http')
    # ctx = InferContext('10.100.37.20:19992', protocol, 'dr_cls', -1, True)
    # ctx = InferContext('10.100.37.20:19992', protocol, 'feibuzhang_cls', -1, True)
    ctx = InferContext('{}:{}'.format(service_ip, service_port), protocol, service_name, -1, True)

    # test_file = '/data/zhangwd/data/examples/dr_deformable/test_label.txt'

    infiles = glob('./xray_data/data/*.png')
    zhengwei_list = []
    lateral_list = []
    error_list = []
    for infile in tqdm(infiles):
        img = get_img(infile)
        result = ctx.run({'images': (img,)}, {'result': (InferContext.ResultFormat.CLASS, 5)}, 1)
#         print(int(result['result'][0][0][0]))
        pos = int(result['result'][0][0][0])
        basename = os.path.basename(infile)
        if pos == 0:
            zhengwei_list.append(basename)
        elif pos == 1:
            lateral_list.append(basename)
        else:
            error_list.append(basename)
    with open('./xray_data/zhengwei.txt', 'w') as f:
        f.write('\n'.join(zhengwei_list))
    with open('./xray_data/lateral.txt', 'w') as f:
        f.write('\n'.join(lateral_list))
    with open('./xray_data/error.txt', 'w') as f:
        f.write('\n'.join(error_list))
    print('frontal cnt:\t{}'.format(len(zhengwei_list)))
    print('lateral cnt:\t{}'.format(len(lateral_list)))
    print('error cnt:\t{}'.format(len(error_list)))

In [4]:
infer_with_trt(None, '10.100.37.20', '19991', 'dr_pos_recog', None)

100%|██████████| 7470/7470 [20:39<00:00,  6.03it/s]

frontal cnt:	2928
lateral cnt:	4285
error cnt:	257
